# Image Classification with VGG

This tutorial will introduce the attribution of image classifiers using VGG11
and ResNet18 on ImageNet. Feel free to replace VGG11 and ResNet18 with any
other version of VGG or ResNet respectively.

## Preparation

First, we install **Zennit**. This includes its dependencies `Pillow`,
`torch` and `torchvision`:

In [ ]:
%pip install zennit

Then, we import necessary modules, classes and functions:

In [ ]:
import logging

import torch
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop
from torchvision.transforms import ToTensor, Normalize
from torchvision.models import vgg11_bn, resnet18

from zennit.attribution import Gradient, SmoothGrad
from zennit.composites import EpsilonGammaBox, EpsilonPlusFlat
from zennit.composites import SpecialFirstLayerMapComposite, NameMapComposite
from zennit.image import imgify, imsave
from zennit.rules import Epsilon, ZPlus, Norm, Pass, Flat
from zennit.types import Convolution, Activation, AvgPool
from zennit.torchvision import VGGCanonizer, ResNetCanonizer

We download an image of the [Dornbusch
Lighthouse](https://en.wikipedia.org/wiki/Dornbusch_Lighthouse) from [Wikimedia
Commons](https://commons.wikimedia.org/wiki/File:2006_09_06_180_Leuchtturm.jpg):

In [ ]:
torch.hub.download_url_to_file(
    'https://upload.wikimedia.org/wikipedia/commons/thumb/8/8b/2006_09_06_180_Leuchtturm.jpg/640px-2006_09_06_181_Leuchtturm.jpg',
    'dornbusch-lighthouse.jpg',
)

We load and prepare the data. The image is resized such that the shorter side
is 256 pixels in size, then center-cropped to `(224, 224)`, converted to a
`torch.Tensor`, and then normalized according the channel-wise mean and
standard deviation of the ImageNet dataset:

In [ ]:
# define the base image transform
transform_img = Compose([
    Resize(256),
    CenterCrop(224),
])
# define the normalization transform
transform_norm = Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
# define the full tensor transform
transform = Compose([
    transform_img,
    ToTensor(),
    transform_norm,
])

# load the image
image = Image.open('dornbusch-lighthouse.jpg')

# transform the PIL image and insert a batch-dimension
data = transform(image)[None]

We can look at the original image and the cropped image:

In [ ]:
# display the original image
display(image)
# display the resized and cropped image
display(transform_img(image))

## VGG11 without BatchNorm

We start with VGG11 without BatchNorm.
First, we initialize the VGG16 model and load the hyperparameters. Set
`pretrained=True` to use the pre-trained model instead of the random one:

In [ ]:
# load the model and set it to evaluation mode
model = vgg11(pretrained=False).eval()

### VGG11 - Sensitivity Analysis
We first compute the Sensitivity-Analysis attribution, which is the squared
gradient, by using the *Gradient* **Attributor**:

In [ ]:
# create the attributor, specifying model
with Gradient(model=model) as attributor:
    # compute the model output and attribution
    output, attribution = attributor(data, target)

print(f'Prediction: {output.argmax(1)[0].item()}')

Visualize the attribution:

In [ ]:
# square and sum over the channels
relevance = (attribution ** 2).sum(1)

# create an image of the visualize attribution the relevance is only
# positive, so we use symmetric=False and an unsigned color-map
img = imgify(relevance, symmetric=False, cmap='hot')

# show the image
display(img)

### VGG11 - SmoothGrad using Attributors
Model-agnostic attribution methods like *SmootGrad* are implemented as *Attributors*.
For these, we simply replace the `Gradient` **Attributor** with the appropriate **Attributor**:

In [ ]:
# create the attributor, specifying model
with SmoothGrad(noise_level=0.1, n_iter=20, model=model) as attributor:
    # compute the model output and attribution
    output, attribution = attributor(data, target)

print(f'Prediction: {output.argmax(1)[0].item()}')

Visualize the attribution:

In [ ]:
# take the absolute and sum over the channels
relevance = attribution.abs().sum(1)

# create an image of the visualize attribution the relevance is only
# positive, so we use symmetric=False and an unsigned color-map
img = imgify(relevance, symmetric=False, cmap='hot')

# show the image
display(img)

### VGG11 - Layer-wise Relevance Propagation (LRP) with EpsilonPlusFlat
We compute the LRP-attribution also using the *Gradient* **Attributor**
together with the `EpsilonPlusFlat` **Composite**:

In [ ]:
# create a composite
composite = EpsilonPlusFlat()

# choose a target class for the attribution (label 437 is lighthouse)
target = torch.eye(1000)[[437]]

# create the attributor, specifying model and composite
with Gradient(model=model, composite=composite) as attributor:
    # compute the model output and attribution
    output, attribution = attributor(data, target)

print(f'Prediction: {output.argmax(1)[0].item()}')

Visualize the attribution:

In [ ]:
# sum over the channels
relevance = attribution.sum(1)

# create an image of the visualize attribution
img = imgify(relevance, symmetric=True, cmap='coldnhot')

# show the image
display(img)

### VGG11 - Layer-wise Relevance Propagation (LRP) with EpsilonGammaBox
We now compute the LRP-attribution with the `EpsilonGammaBox` **Composite**.
The `EpsilonGammaBox` **Composite** uses the `ZBox` rule, which needs as
arguments the lowest and highest possible values.

In [ ]:
# use the VGG-specific canonizer (alias for SequentialMergeBatchNorm, only
# needed with batch-norm)
canonizer = VGGCanonizer()

# the EpsilonGammaBox composite needs the lowest and highest values, which are
# here for ImageNet 0. and 1. with a different normalization for each channel
low, high = transform_norm(torch.tensor([[[[[0.]]] * 3], [[[[1.]]] * 3]]))

# create a composite, specifying arguments and the canonizers, if any
composite = EpsilonGammaBox(low=low, high=high, canonizers=[canonizer])

# choose a target class for the attribution (label 437 is lighthouse)
target = torch.eye(1000)[[437]]

# create the attributor, specifying model and composite
with Gradient(model=model, composite=composite) as attributor:
    # compute the model output and attribution
    output, attribution = attributor(data, target)

print(f'Prediction: {output.argmax(1)[0].item()}')

Visualize the attribution:

In [ ]:
# sum over the channels
relevance = attribution.sum(1)

# create an image of the visualize attribution
img = imgify(relevance, symmetric=True, cmap='coldnhot')

# show the image
display(img)

### More Visualization
We can try out different color-maps by either using another built-in color map, or using the color-map specification language:

In [ ]:
print('Built-in color-map bwr')
display(imgify(relevance, symmetric=True, cmap='bwr'))

print('CMLS code for color-map from cyan to grey to purple')
display(imgify(relevance, symmetric=True, cmap='0ff,444,f0f'))

print('CMSL code for grey scale for negative and red for positive values')
display(imgify(relevance, symmetric=True, cmap='fff,000,f00'))

To directly save the visualized attribution, we can use `imsave` instead:

In [ ]:
# directly save the visualized attribution
imsave('attrib-1.png', relevance, symmetric=True, cmap='bwr')

## VGG11 with BatchNorm

VGG11 with BatchNorm requires a canonizer for LRP to function properly.
We initialize the VGG11-bn model and load the hyperparameters. Set
`pretrained=True` to use the pre-trained model instead of the random one:

In [ ]:
# load the model and set it to evaluation mode
model = vgg11_bn(pretrained=False).eval()

### VGG11-bn - Layer-wise Relevance Propagation (LRP) with EpsilonGammaBox
We now compute the LRP-attribution with the `EpsilonGammaBox` **Composite**.

In [ ]:
# use the VGG-specific canonizer (alias for SequentialMergeBatchNorm, only
# needed with batch-norm)
canonizer = VGGCanonizer()

# the EpsilonGammaBox composite needs the lowest and highest values, which are
# here for ImageNet 0. and 1. with a different normalization for each channel
low, high = transform_norm(torch.tensor([[[[[0.]]] * 3], [[[[1.]]] * 3]]))

# create a composite, specifying arguments and the canonizers, if any
composite = EpsilonGammaBox(low=low, high=high, canonizers=[canonizer])

# choose a target class for the attribution (label 437 is lighthouse)
target = torch.eye(1000)[[437]]

# create the attributor, specifying model and composite
with Gradient(model=model, composite=composite) as attributor:
    # compute the model output and attribution
    output, attribution = attributor(data, target)

print(f'Prediction: {output.argmax(1)[0].item()}')

Visualize the attribution:

In [ ]:
# sum over the channels
relevance = attribution.sum(1)

# create an image of the visualize attribution
img = imgify(relevance, symmetric=True, cmap='coldnhot')

# show the image
display(img)

### VGG11-bn - Layer-wise Relevance Propagation (LRP) with Custom NameMapComposite
We demonstrate how rules can be assigned on a name-base using the abstract
`NameMapComposite`. Note that this particular case will only work with
VGG11-bn, so we redefine the model:

In [ ]:
# load the model and set it to evaluation mode
model = vgg11_bn(pretrained=False).eval()

We can take a look at all leaf modules by doing the following:

In [ ]:
leaves = [
    (name, module)
    for name, module in model.named_modules()
    if not [*islice(module.children(), 1)]
]
maxlen = max(len(name) for name, _ in leaves)
print('\n'.join(f'{name:{maxlen}s}: {module}' for name, module in leaves))

We can look at the architecture and the assign the rules directly to the names:

In [ ]:
name_map = [
    (['features.0'], Flat()),  # Conv2d
    (['features.1'], Pass()),  # BatchNorm2d
    (['features.2'], Pass()),  # ReLU
    (['features.3'], Norm()),  # MaxPool2d
    (['features.4'], ZPlus()),  # Conv2d
    (['features.5'], Pass()),  # BatchNorm2d
    (['features.6'], Pass()),  # ReLU
    (['features.7'], Norm()),  # MaxPool2d
    (['features.8'], ZPlus()),  # Conv2d
    (['features.9'], Pass()),  # BatchNorm2d
    (['features.10'], Pass()),  # ReLU
    (['features.11'], ZPlus()),  # Conv2d
    (['features.12'], Pass()),  # BatchNorm2d
    (['features.13'], Pass()),  # ReLU
    (['features.14'], Norm()),  # MaxPool2d
    (['features.15'], ZPlus()),  # Conv2d
    (['features.16'], Pass()),  # BatchNorm2d
    (['features.17'], Pass()),  # ReLU
    (['features.18'], ZPlus()),  # Conv2d
    (['features.19'], Pass()),  # BatchNorm2d
    (['features.20'], Pass()),  # ReLU
    (['features.21'], Norm()),  # MaxPool2d
    (['features.22'], ZPlus()),  # Conv2d
    (['features.23'], Pass()),  # BatchNorm2d
    (['features.24'], Pass()),  # ReLU
    (['features.25'], ZPlus()),  # Conv2d
    (['features.26'], Pass()),  # BatchNorm2d
    (['features.27'], Pass()),  # ReLU
    (['features.28'], Norm()),  # MaxPool2d
    (['avgpool'], Norm()),  # AdaptiveAvgPool2d
    (['classifier.0'], Epsilon()),  # Linear
    (['classifier.1'], Pass()),  # ReLU
    (['classifier.2'], Pass()),  # Dropout
    (['classifier.3'], Epsilon()),  # Linear
    (['classifier.4'], Pass()),  # ReLU
    (['classifier.5'], Pass()),  # Dropout
    (['classifier.6'], Epsilon()),  # Linear
]

We can use that name-map to create the composite:

In [ ]:
# use the VGG-specific canonizer (alias for SequentialMergeBatchNorm, only
# needed with batch-norm)
canonizer = VGGCanonizer()

# create a composite, specifying arguments and the canonizers, if any
composite = NameMapComposite(
    name_map=name_map,
    canonizers=[canonizer],
)

# choose a target class for the attribution (label 437 is lighthouse)
target = torch.eye(1000)[[437]]

# create the attributor, specifying model and composite
with Gradient(model=model, composite=composite) as attributor:
    # compute the model output and attribution
    output, attribution = attributor(data, target)

print(f'Prediction: {output.argmax(1)[0].item()}')

Visualize the attribution:

In [ ]:
# sum over the channels
relevance = attribution.sum(1)

# create an image of the visualize attribution
img = imgify(relevance, symmetric=True, cmap='coldnhot')

# show the image
display(img)

## ResNet18
Then, we initialize the ResNet18 model and load the hyperparameters. Set
`pretrained=True` to use the pre-trained model instead of the random one:

In [ ]:
# load the model and set it to evaluation mode
model = resnet18(pretrained=False).eval()

### ResNet18 - LRP with EpsilonPlusFlat
Compute the LRP-attribution using the ``EpsilonPlusFlat`` **Composite**:

In [ ]:
# use the ResNet-specific canonizer
canonizer = ResNetCanonizer()

# create a composite, specifying the canonizers, if any
composite = EpsilonPlusFlat(canonizers=[canonizer])

# choose a target class for the attribution (label 437 is lighthouse)
target = torch.eye(1000)[[437]]

# create the attributor, specifying model and composite
with Gradient(model=model, composite=composite) as attributor:
    # compute the model output and attribution
    output, attribution = attributor(data, target)

print(f'Prediction: {output.argmax(1)[0].item()}')

Visualize the attribution:

In [ ]:
# sum over the channels
relevance = attribution.sum(1)

# create an image of the visualize attribution
img = imgify(relevance, symmetric=True, cmap='coldnhot')

# show the image
display(img)

### ResNet18 - LRP with a Custom LayerMapComposite
In this demonstration, we want to ignore the BatchNorm layers and attribute the
residual connections not by contribution but equally. This setup requires no **Canonizer**, but a special Composite which ignores the BatchNorm modules for LRP by assigning them the `Pass` rule.

In [ ]:
# the ZBox rule needs the lowest and highest values, which are here for
# ImageNet 0. and 1. with a different normalization for each channel
low, high = transform_norm(torch.tensor([[[[[0.]]] * 3], [[[[1.]]] * 3]]))

# create a composite, specifying the canonizers, if any
composite = SpecialFirstLayerMapComposite(
    # the layer map is a list of tuples, where the first element is the target
    # layer type, and the second is the rule template
    layer_map=[
        (Activation, Pass()),  # ignore activations
        (AvgPool, Norm()),  # normalize relevance for any AvgPool
        (Convolution, ZPlus()),  # any convolutional layer
        (Linear, Epsilon(epsilon=1e-6))  # this is the dense Linear, not any
        (BatchNorm, Pass())  # ignore BatchNorm
    ]
    # the first map is only used once, to the first module which applies to the
    # map, i.e. here the first layer of type AnyLinear
    first_map = [
        (AnyLinear, ZBox(low, high))
    ]
)

# choose a target class for the attribution (label 437 is lighthouse)
target = torch.eye(1000)[[437]]

# create the attributor, specifying model and composite
with Gradient(model=model, composite=composite) as attributor:
    # compute the model output and attribution
    output, attribution = attributor(data, target)

print(f'Prediction: {output.argmax(1)[0].item()}')

Visualize the attribution:

In [ ]:
# sum over the channels
relevance = attribution.sum(1)

# create an image of the visualize attribution
img = imgify(relevance, symmetric=True, cmap='coldnhot')

# show the image
display(img)